In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE101931'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Acute Exercise Bout Effects on GH and IGF1 in Prediabetic and Healthy African Americans"
!Series_summary	"The incidence of pre-diabetes (PD) and Type-2 Diabetes Mellitus (T2D) is a worldwide epidemic. African American (AA) individuals are disproportionately more likely to become diabetic than other ethnic groups. Over the long-term, metabolic complications related to diabetes result in significant alterations in growth hormone (GH) and insulin-like growth factor-1 (IGF-1). Considering the limited exercise-related studies in the area of gene expression changes with disease progression, the objective of this study was to examine differences in exercise-induced gene expression related to the GH and IGF-1 pathways in peripheral blood mononuclear cells (PBMCs) of healthy (CON) and PD AA individuals. Design: Ten subjects [5 PD (age=35±9.3 yr, BMI=32.1±4.0, FBG=101.8±1.3 mg/dl) and 5 CON (age=31±9.4 yr, BMI=29.4±5.2, FBG=82.8±9.7 mg/dl)] had blood drawn 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = 1
gender_row = None  # Gender information is not available in the provided dataset
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Function to convert 'Physical_Exercise_Response'
def convert_trait(value):
    response_value = value.split(': ')[1]
    if response_value == 'Healthy':
        return 0
    elif response_value == 'Prediabetic':
        return 1
    else:
        return None

# Function to convert 'age'
def convert_age(value):
    age_value = value.split(': ')[1]
    try:
        return float(age_value)
    except ValueError:
        return None

save_cohort_info('GSE101931', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE101931.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2719035': [0.0, 24.0], 'GSM2719036': [0.0, 24.0], 'GSM2719037': [0.0, 24.0], 'GSM2719038': [0.0, 24.0], 'GSM2719039': [0.0, 24.0], 'GSM2719040': [0.0, 24.0], 'GSM2719041': [0.0, 24.0], 'GSM2719042': [0.0, 24.0], 'GSM2719043': [0.0, 26.0], 'GSM2719044': [0.0, 26.0], 'GSM2719045': [0.0, 26.0], 'GSM2719046': [0.0, 26.0], 'GSM2719047': [0.0, 45.0], 'GSM2719048': [0.0, 45.0], 'GSM2719049': [0.0, 45.0], 'GSM2719050': [0.0, 45.0], 'GSM2719051': [0.0, 37.0], 'GSM2719052': [0.0, 37.0], 'GSM2719053': [0.0, 37.0], 'GSM2719054': [0.0, 37.0], 'GSM2719055': [1.0, 41.0], 'GSM2719056': [1.0, 41.0], 'GSM2719057': [1.0, 41.0], 'GSM2719058': [1.0, 41.0], 'GSM2719059': [1.0, 33.0], 'GSM2719060': [1.0, 33.0], 'GSM2719061': [1.0, 35.0], 'GSM2719062': [1.0, 35.0], 'GSM2719063': [1.0, 35.0], 'GSM2719064': [1.0, 35.0], 'GSM2719065': [1.0, 44.0], 'GSM2719066': [1.0, 44.0], 'GSM2719067': [1.0, 44.0], 'GSM2719068': [1.0, 44.0], 'GSM2719069': [1.0, 20.0], 'GSM2719070': [1.0, 20.0], 'GSM2719071': [1.0, 20.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Physical_Exercise_Response/gene_data/GSE101931.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Physical_Exercise_Response')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE101931', './preprocessed/Physical_Exercise_Response/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Physical_Exercise_Response/GSE101931.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Physical_Exercise_Response', the least common label is '1.0' with 18 occurrences. This represents 47.37% of the dataset.
The distribution of the feature 'Physical_Exercise_Response' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.0
  50% (Median): 35.0
  75%: 41.0
Min: 20.0
Max: 45.0
The distribution of the feature 'Age' in this dataset is fine.

